### 3.4 Clustering

Python program to evaluate performance using clustering on the datasets obtained in Task 3.1.

In [ ]:
%load_ext autoreload
%autoreload 2

import bamboo.gwp_pipeline as gwp
import bamboo.star_pipeline as star
from bamboo.processing import read_csv, split_features_and_target, get_feature_and_target_names
from bamboo.analysis import *

SEED = 42

GWP_CSV = 'datasets/gwp_assessment.csv'
STAR_CSV = 'datasets/star_assessment.csv'

### Load Datasets

In [ ]:
# Load and process GWP from csv
gwp_data = read_csv(GWP_CSV)
gwp_dataset = gwp.gwp_pipeline(gwp_data)

# Load and process Star from csv
star_data = read_csv(STAR_CSV)
star_dataset = star.star_pipeline(star_data)

#### Markdown Question: 
**Question**: Between clustering and classification algorithms, which one performed better for star dataset. Justify your answer.